In [1]:
#to export a notebook to html use:
# >ipython nbconvert --to html Process-GitHub-Repos.ipynb

from pandas import DataFrame, Series
import pandas as pd
import matplotlib.pyplot as plt
import string
from datetime import datetime, date, time

import re
%pylab inline
import numpy as np
import scipy as sc
import scipy.stats as stats
from scipy.optimize import leastsq

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [2]:
from collections import Counter

data_file = './datasets/msnbc990928.seq'
file = open(data_file, 'r')
state_cnt = Counter()
overall_state_trans = Counter()
list_state_trans = []
list_state_visit = []
for index,line in enumerate(file):
    if index<7:
        continue
#   print index," : ",line.split()
    traj = line.split()
    if len(traj)>500:
        continue
        
    state_cnt = Counter()
    for state in traj:
        state_cnt[int(state)]+=1
    list_state_visit.append(state_cnt)
    
    traj = ['0']+line.split()
    state_trans_cnt = Counter()
    for prevstate,state in zip(traj[:-1],traj[1:]):
        state_trans_cnt[prevstate+">"+state]+=1
        overall_state_trans[prevstate+">"+state]+=1

    list_state_trans.append(state_trans_cnt)

In [3]:
df_state_visit = DataFrame(list_state_visit)
df_state_visit = df_state_visit.fillna(0)
df_state_visit.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,5,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
df_state_visit.shape

(989782, 17)

In [8]:
import sklearn.cluster
from sklearn.cluster import MiniBatchKMeans, KMeans

def do_kmeans(df):
    # Convert DataFrame to matrix
    mat = df.as_matrix()
    # Using sklearn
    k_means = sklearn.cluster.KMeans(n_clusters=10, init='k-means++',n_jobs=2)
    k_means.fit(mat)
    # Get cluster assignment labels
    return k_means.labels_,k_means.cluster_centers_

In [9]:
k_means_labels,k_means_cluster_centers = do_kmeans(df_state_visit)

In [10]:
df_res = DataFrame(df_state_visit)
df_res['k_means_labels'] = k_means_labels
df_res.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,k_means_labels
0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,5,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,8
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
cluster_counts = df_res.k_means_labels.value_counts()
for i,s in zip(cluster_counts.index,cluster_counts):
    print i," : ",s," || ",k_means_cluster_centers[i]
    print

4  :  836345  ||  [ 0.49818317  0.21126449  0.19822083  0.22817617  0.0674602   0.41042512
  0.18824289  0.1204204   0.19957793  0.10209064  0.07837196  0.23531318
  0.21765898  0.14392984  0.04933251  0.0101836   0.01194124]

1  :  44590  ||  [ 8.80580848  0.76108993  0.31551917  0.43570307  0.15967706  0.36958959
  0.49784705  0.08261942  0.0566719   0.25821933  0.31314196  0.66382597
  0.02653061  0.46721238  0.06144876  0.03132989  0.0739852 ]

3  :  36040  ||  [  2.16897891e-01   2.22086570e-01   5.80743618e-02   2.25083241e-01
   5.04439512e-02   1.73723640e-01   1.89178690e-01   8.84805771e+00
   1.89289678e-01   7.41398446e-02   5.72974473e-02   7.98002220e-02
   1.54078801e-01   1.27608213e-01   2.01442841e-02   2.19200888e-03
   1.29578246e-02]

2  :  30394  ||  [ 0.96657235  0.37533724  0.11893795  0.23409226  0.07603474  0.18605646
  0.35622162  0.12926893  0.13134171  0.07860104  0.10446141  0.24741725
  0.79295256  7.44600908  0.04372574  0.03536882  0.01954333]

0  :  29

In [49]:
df_k_means_centers = DataFrame(k_means_cluster_centers)
df_k_means_centers[:1].round(0)

AttributeError: 'DataFrame' object has no attribute 'round'

In [50]:
print df_k_means_centers[1]

KeyError: 16

In [ ]:
df_k_means_centers[1].round

In [18]:
df_res[df_res['k_means_labels']==2]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,k_means_labels
38,0,0,0,2,0,0,8,0,0,0,0,0,11,3,0,0,0,2
44,0,0,0,0,0,0,0,1,0,0,0,0,4,5,0,0,0,2
68,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,2
74,3,3,0,0,0,0,0,0,0,0,0,0,0,12,0,0,0,2
104,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,2
116,4,1,0,0,0,0,0,0,0,2,0,1,0,6,0,0,0,2
135,0,0,0,0,0,0,0,0,0,0,0,0,3,16,0,0,0,2
165,3,0,0,0,0,0,0,0,0,0,0,0,0,17,0,0,0,2
166,3,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,2
215,0,0,0,2,0,0,0,3,0,0,0,0,0,5,0,0,0,2


In [174]:
from sklearn import metrics
from sklearn.metrics import pairwise_distances
metrics.silhouette_score(df_state_visit.as_matrix(), k_means_labels, metric='euclidean',sample_size =10000)

0.54214805215958339

In [175]:
from sklearn import metrics
from sklearn.metrics import pairwise_distances
metrics.silhouette_score(df_state_visit.as_matrix(), k_means_labels, metric='euclidean',sample_size =20000)

0.54358861599043462